This notebook inspects optimal hyperparameters for classification models finetuning.

In [1]:

import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()

In [1]:
label_set = ['Negative', 'Positive', 'Neutral']
STR_TO_NUM = {k: i for i, k in enumerate(label_set)}
NUM_TO_STR = {i:k for i, k in enumerate(label_set)}

import pandas as pd
df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
df = df[["sentence", "label", "split"]].rename(columns={"sentence": "text", "label":"labels"})
train = df[df.split=="train"].drop(columns=["split"])
dev = df[df.split=="dev"].drop(columns=["split"])
test = df[df.split=="test"].drop(columns=["split"])


In [9]:

def train_model(train_df, model_name, model_type, batch_size, NUM_EPOCHS=5):
    from simpletransformers.classification import ClassificationModel
    model_args = {
        "num_train_epochs": NUM_EPOCHS,
        "learning_rate": 1e-5,
        "overwrite_output_dir": True,
        "train_batch_size": batch_size,
        "no_save": True,
        "no_cache": True,
        "overwrite_output_dir": True,
        "save_steps": -1,
        "max_seq_length": 512,
        "silent": True
    }

    model = ClassificationModel(
        model_type, model_name,
        num_labels = 3,
        use_cuda = True,
        args = model_args
    )
    model.train_model(train_df)
    return model

def eval_model(model, test_df):
    y_true_enc = test_df.labels
    y_pred_enc = model.predict(test_df.text.tolist())[0]

    y_true = [NUM_TO_STR[i] for i in y_true_enc]
    y_pred = [NUM_TO_STR[i] for i in y_pred_enc]
    from sklearn.metrics import f1_score
    microF1 = f1_score(y_true, y_pred, labels=["Positive", "Neutral", "Negative"], average ="micro")
    macroF1 = f1_score(y_true, y_pred, labels=["Positive", "Neutral", "Negative"], average ="macro")

    return {"microF1": microF1, 
            "macroF1": macroF1,
            "y_true": y_true,
            "y_pred": y_pred}


for epoch in [3, 5, 9, 15]:
    for batch in [8, 16]:
        for modeltype, modelname in zip(
            ["xlmroberta", "xlmroberta", "bert", "electra"],
            ["xlm-roberta-base", "xlm-roberta-large", "EMBEDDIA/crosloengual-bert", "classla/bcms-bertic"]):
            model = train_model(train, modelname, modeltype,batch,NUM_EPOCHS=epoch)
            stats = eval_model(model, dev)
            with open("results3.csv", "a") as f:
                f.write(f"{modelname},{modeltype},{epoch},{batch},{stats['macroF1']}\n")


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [7]:
eval_model(model, dev)

{'microF1': 0.78,
 'macroF1': 0.7610070962584262,
 'y_true': ['Neutral',
  'Positive',
  'Neutral',
  'Positive',
  'Neutral',
  'Neutral',
  'Positive',
  'Negative',
  'Negative',
  'Neutral',
  'Neutral',
  'Neutral',
  'Neutral',
  'Negative',
  'Neutral',
  'Negative',
  'Negative',
  'Negative',
  'Negative',
  'Negative',
  'Negative',
  'Negative',
  'Negative',
  'Positive',
  'Negative',
  'Positive',
  'Negative',
  'Negative',
  'Negative',
  'Positive',
  'Negative',
  'Neutral',
  'Negative',
  'Negative',
  'Positive',
  'Positive',
  'Neutral',
  'Negative',
  'Positive',
  'Negative',
  'Negative',
  'Negative',
  'Positive',
  'Positive',
  'Negative',
  'Positive',
  'Negative',
  'Positive',
  'Negative',
  'Positive',
  'Negative',
  'Negative',
  'Neutral',
  'Neutral',
  'Neutral',
  'Negative',
  'Neutral',
  'Negative',
  'Negative',
  'Negative',
  'Neutral',
  'Negative',
  'Neutral',
  'Positive',
  'Positive',
  'Negative',
  'Negative',
  'Neutral',
  'Pos